Extract Data 1.1 

For “Credit Card System,” create a Python and PySpark SQL program to read/extract the following JSON files according to the specifications found in the mapping document.

In [3]:
# Import neccessary Pyspark modules
import pyspark
from pyspark.sql import SparkSession
# Create or retrieve an existing SparkSession.
# This is the entry point to use DataFrame and SQL functionality in PySpark.
# The 'appName' is used to name your Spark application, which helps when monitoring jobs in Spark UI.
spark = SparkSession.builder.appName('Credit Card Data Loader').getOrCreate()
# Load the 'cdw_sapp_branch.json' JSON file into a DataFrame.
# Load the 'cdw_sapp_credit.json' JSON file into a DataFrame.
# Load the 'cdw_sapp_customer.json' JSON file into a DataFrame.
# The 'option("multiline", "true")' tells Spark to handle JSON entries that span multiple lines.
# This is useful when each record is formatted across several lines for readability.
df_branch = spark.read.option("multiline", "true").json(r"C:\Users\malik.alston\Desktop\Data\Capstone\Credit Card Dataset Overview\cdw_sapp_branch.json")
df_cc = spark.read.option("multiline", "true").json(r"C:\Users\malik.alston\Desktop\Data\Capstone\Credit Card Dataset Overview\cdw_sapp_credit.json")
df_customer = spark.read.option("multiline", "true").json(r"C:\Users\malik.alston\Desktop\Data\Capstone\Credit Card Dataset Overview\cdw_sapp_customer.json")
# The below lines are currently commented out but can be used to visually inspect
# The contents of each DataFrame. They display the top 20 rows by default in a tabular format.
# df_customer.show()
# df_branch.show()
# df_cc.show()

Transform Data

In [ ]:
from pyspark.sql.functions import initcap, lower, concat_ws, format_string, col, lpad, lit, concat, when
# initcap()	Capitalize first letter of each word, lower() Convert string to lowercase, concat_ws()	Concatenate with separator, format_string()	String formatting
# col()	Reference to a column, lpad() Left-pad a string, lit()	Add a constant/literal column, concat()	Concatenate columns (no separator), when() Conditional logic
df_customer = df_customer.withColumn("FIRST_NAME", initcap("FIRST_NAME"))
df_customer = df_customer.withColumn("LAST_NAME", initcap("LAST_NAME"))
df_customer = df_customer.withColumn("MIDDLE_NAME", lower("MIDDLE_NAME"))
df_customer = df_customer.withColumn("ADDRESS", concat_ws(",", "STREET_NAME", "APT_NO"))
df_customer = df_customer.withColumn("CUST_PHONE",concat(lit("(XXX)"),col("CUST_PHONE").substr(1, 3),lit("-"),col("CUST_PHONE").substr(4, 4)))

df_branch = df_branch.withColumn("BRANCH_PHONE",concat(lit("(XXX)"),col("BRANCH_PHONE").substr(1, 3),lit("-"),col("BRANCH_PHONE").substr(4, 4)))
df_branch = df_branch.withColumn("BRANCH_ZIP", when(col("BRANCH_ZIP").isNull(), lit("99999")).otherwise(col("BRANCH_ZIP")))

df_cc = df_cc.withColumn("TIMEID", format_string("%04d%02d%02d", col("YEAR"), col("MONTH"), col("DAY")))
# The below lines are currently commented out but can be used to visually inspect
# The contents of each DataFrame. They display the top 20 rows by default in a tabular format.
#df_customer.show()
#df_branch.show()
#df_cc.show()

Load Data 1.2

In [ ]:
# Define the JDBC MySQL connection URL
mysql_url = "jdbc:mysql://localhost:3306/creditcard_capstone"
# Define the configuration dictionary for MySQL connection
mysql_config = {
    "user": "root",   # MySQL username
    "password": "password",   # MySQL password
    "driver": "com.mysql.cj.jdbc.Driver"   # JDBC driver for MySQL
}
# Write the DataFrame 'df_customer' to the MySQL table 'CDW_SAPP_CUSTOMER'
# 'mode="append"' ensures that data will be added to the existing table without overwriting it
df_customer.write.jdbc(
    url=mysql_url,   # JDBC connection URL
    table="CDW_SAPP_CUSTOMER",   # Destination table name in MySQL
    mode="append",  # use 'append' if you want to preserve existing data
    properties=mysql_config   # Pass the MySQL configuration
)
# Write the DataFrame 'df_branch' to the MySQL table 'CDW_SAPP_BRANCH'
# Again, 'append' mode is used to add data without deleting existing records
df_branch.write.jdbc(
    url=mysql_url,
    table="CDW_SAPP_BRANCH",
    mode="append",
    properties=mysql_config
)
# Write the DataFrame 'df_cc' to the MySQL table 'CDW_SAPP_CREDIT_CARD'
# This transfers credit card transaction data to the specified table
df_cc.write.jdbc(
    url=mysql_url,
    table="CDW_SAPP_CREDIT_CARD",
    mode="append",
    properties=mysql_config
)
# Confirmation message to indicate all DataFrames have been written to MySQL
print("All DataFrames transferred to MySQL.")

All DataFrames transferred to MySQL.


df_customer.printSchema()
df_branch.printSchema()
df_cc.printSchema()